In [ ]:
#word2vec tutorial: https://rare-technologies.com/word2vec-tutorial/

In [ ]:
import gensim, logging
# the model is organized like this: word = embeddings
small_model = gensim.models.KeyedVectors.load_word2vec_format('small-embeddings.txt', binary=False)


In [ ]:
#to see the embeddings of a word, you just do:

print (small_model["clinton"])
print (small_model["obama"])

In [ ]:
#finding similar words

small_model.wv.most_similar(positive=['Obama'])

In [ ]:
# get relatedness

print (small_model.wv.similarity('clinton', 'clinton'))
print (small_model.wv.similarity('clinton', 'obama'))

In [ ]:
# you can represent the meaning of an article, by the average of their embeddings
# let's compute the embeddings for an article

dataset = codecs.open("dataset.tsv", "r", "utf-8").read().strip().split("\n")

article = dataset[4].split("\t")[3]

In [ ]:
import codecs, nltk, string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

exclude = set(string.punctuation)
stop_word_list = stopwords.words('english')

# input should be a string - we need a simple pipeline for getting word embeddings
def nlp_simple_pipeline(text):
    
    #it depends if the words have been lowercased or not
    text = text.lower()
    
    text = nltk.word_tokenize(text)
        
    text = [token for token in text if token not in exclude and token.isalpha()]
    
    text = [token for token in text if token not in stop_word_list]

    return text

In [ ]:
# for each word, load embeddings
for word in cleaned_article:
    print (word)
    embed_word = small_model[word]


In [ ]:
# handling exceptions
for word in cleaned_article:
    try:
        embed_word = small_model[word]
    except KeyError:
        print (word)
        continue

In [ ]:
# representing the meaning of an article as a single embedding

def article_embedding(cleaned_article):
    
    article_embedd = []
    # for each word in the article, you take the embeddings
    for word in cleaned_article:
        try:
            embed_word = small_model[word]
            article_embedd.append(embed_word)
        except KeyError:
            continue
    
    # average vectors of all words
    avg = [float(sum(col))/len(col) for col in zip(*article_embedd)]
    return avg

homework: write a single function that takes an article, cleans the article and creates the article-embeddings

In [ ]:
#dictionary of article embeddings
articles_embeddings = {}

# you can limit this to the first 100 articles with
#for k in range(len(dataset[:100])):

for k in range(len(dataset[:1000])):
    article = dataset[k]
    title = article.split("\t")[1]
    cleaned_article = nlp_simple_pipeline(article)
    embed_art = article_embedding(cleaned_article)
    articles_embeddings[str(k)] = {"title":title,"embed":embed_art}


In [ ]:
our_article_embedd = articles_embeddings["4"]["embed"]
print (our_article_embedd)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity


ranking = []

for article,values in articles_embeddings.items():
    similarity_score = cosine_similarity(values["embed"],our_article_embedd)[0][0]
    ranking.append([values["title"],similarity_score])

In [ ]:
ranking.sort(key=lambda x: x[1],reverse=True)

In [ ]:
for el,score in ranking[:10]:
    print (el,score)